In [1]:
%pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain dotenv langchain_huggingface

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv

# Load from .env
load_dotenv()

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# Basic for Retrievel - Find the similarities between query and document


In [3]:
# Documents
question = "Who is the GOAT in football?"
document = "The GOAT in football is widely considered to be Lionel Messi, who has won multiple Ballon d'Or awards and has set numerous records throughout his career."

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

embd = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
print(len(query_result), query_result[:10])


384 [0.015595120377838612, 0.037695035338401794, -0.10810225456953049, -0.06267601996660233, 0.018530840054154396, 0.07376108318567276, 0.06142119690775871, 0.07251938432455063, 0.11813592165708542, 0.05434219911694527]


In [5]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.6851970874359161


# Load Documents - Indexing - Save to VectorDB


In [6]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://vnexpress.net/tau-chien-nga-bi-to-bat-radar-chia-vu-khi-vao-luc-luong-dan-mach-4947092.html",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("Normal")
        )
    ),
)
blog_docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
blog_docs[0].page_content

'"Chúng tôi ghi nhận một vài sự việc tại các eo biển của Đan Mạch, trong đó chiến hạm Nga bật radar để theo dõi tàu hải quân và trực thăng không quân của chúng tôi, cũng như trực tiếp chĩa vũ khí vào lực lượng này", Giám đốc Cơ quan Tình báo Quốc phòng Đan Mạch Thomas Ahrenkiel phát biểu tại cuộc họp báo hôm 3/10.Quan chức Đan Mạch không nêu tên chiến hạm Nga, cũng như không đề cập rõ là con tàu sử dụng radar cảnh giới hay radar điều khiển hỏa lực trong các vụ chạm trán.Radar cảnh giới có tầm hoạt động và khả năng theo dõi tốt hơn radar định vị hàng hải thông thường, giúp chiến hạm nắm bắt tình huống xung quanh và phát hiện mối đe dọa từ xa. Sử dụng hệ thống này không đồng nghĩa với tàu chiến đang ở trạng thái sẵn sàng khai hỏa.Trong khi đó, radar điều khiển hỏa lực được dùng để định vị chính xác đối phương, cung cấp tham số dẫn đường cho tên lửa và đạn có điều khiển. Bật radar hỏa lực và khóa mục tiêu thường chỉ diễn ra khi một bên sẵn sàng khai hỏa trong chiến đấu, đồng thời bị coi l

In [8]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

In [9]:
splits

[Document(metadata={'source': 'https://vnexpress.net/tau-chien-nga-bi-to-bat-radar-chia-vu-khi-vao-luc-luong-dan-mach-4947092.html'}, page_content='"Chúng tôi ghi nhận một vài sự việc tại các eo biển của Đan Mạch, trong đó chiến hạm Nga bật radar để theo dõi tàu hải quân và trực thăng không quân của chúng tôi, cũng như trực tiếp chĩa vũ khí vào lực lượng này", Giám đốc Cơ quan Tình báo Quốc phòng Đan Mạch Thomas Ahrenkiel phát biểu tại cuộc họp báo hôm 3/10.Quan chức Đan Mạch không nêu tên chiến hạm Nga, cũng'),
 Document(metadata={'source': 'https://vnexpress.net/tau-chien-nga-bi-to-bat-radar-chia-vu-khi-vao-luc-luong-dan-mach-4947092.html'}, page_content='hôm 3/10.Quan chức Đan Mạch không nêu tên chiến hạm Nga, cũng như không đề cập rõ là con tàu sử dụng radar cảnh giới hay radar điều khiển hỏa lực trong các vụ chạm trán.Radar cảnh giới có tầm hoạt động và khả năng theo dõi tốt hơn radar định vị hàng hải thông thường, giúp chiến hạm nắm bắt tình huống xung quanh và phát hiện mối đe d

In [10]:
from langchain_community.vectorstores import Chroma

# Create and save to VectorDB
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))

# Create retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [11]:
retriever.get_relevant_documents("Vladimir Putin")

C:\Users\Admin\AppData\Local\Temp\ipykernel_8368\3564590849.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("Vladimir Putin")


[Document(metadata={'source': 'https://vnexpress.net/tau-chien-nga-bi-to-bat-radar-chia-vu-khi-vao-luc-luong-dan-mach-4947092.html'}, page_content='hành cuộc tấn công kiểu "chiến tranh lai" nhằm vào châu Âu.Tổng thống Vladimir Putin ngày 2/10 nói đùa rằng ông sẽ "không cho máy bay không người lái bay qua Đan Mạch nữa", đồng thời mô tả ý tưởng rằng Nga muốn nhắm mục tiêu vào một thành viên NATO là "vô lý".Tàu quân sự Nga thường xuyên đi qua các eo biển của Đan Mạch, vốn là tuyến hàng hải quốc tế nhộn nhịp.')]

# Generation


In [12]:
from langchain.prompts import ChatPromptTemplate

template ="""Answer the question based on the context below:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based on the context below:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [27]:
%pip install langchain_google_genai

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 8.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.6 MB ? eta -:--:--
   ----------- ---------------------------- 1.3/4.6 MB 11.2 MB/s eta 0:00:01
   ----------------------------- ---------- 3.4/4.6 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 4.6/4.6 MB 9.0 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.67.1
    Uninstalling grpcio-1.67.1:
      Successfully uninstalled grpcio-1.67.1
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.16.2 requires ml-dtypes~=0.3.1, but you have ml-dtypes 0.5.3 which is incompatible.
tensorflow-intel 2.16.2 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 6.32.1 which is incompatible.
tensorflow-intel 2.16.2 requires tensorboard<2.17,>=2.16, but you have tensorboard 2.20.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key="XXXXXXXXXXXXXXXXXXXXXXX")

In [30]:
chain = prompt | llm

In [31]:
chain.invoke({"context":retriever.get_relevant_documents("Nga"),"question":"Nga đã làm gì với Đan Mạch?"})

AIMessage(content='Theo tài liệu, tàu chiến Nga đã:\n*   Nhiều lần áp sát tàu Đan Mạch, gây nguy cơ va chạm trực diện khi đi qua các eo biển nối liền Biển Baltic với Biển Bắc.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--93d9506d-cecf-4842-93ce-731439af96bb-0', usage_metadata={'input_tokens': 181, 'output_tokens': 777, 'total_tokens': 958, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 730}})